In [1]:
#import relevant libraries
import os
from scipy import stats

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import dabest
import NLCLIMB
import NLMATH
import itertools
from datetime import datetime
date = datetime.today().strftime('%Y%m%d')
from statistics import mean
from textwrap import wrap

import dabest_jck
import plotly.express as px 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.graph_objects import Layout
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

#NOTE: SUPPRESSES WARNINGS!

import warnings


warnings.simplefilter(action="ignore", category=RuntimeWarning)
warnings.simplefilter(action="ignore", category=UserWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)


In [12]:
#initial file processing
computer1 = "C:\\Users\\User"
computer2 = "C:\\Users\\lnico"
homecomp = "D:"
filedir = "\\ACC Lab Dropbox\\ACC Lab\\Nicole Lee\\Data Compilation\\Falling_New\\"
openPath = homecomp + filedir
files = os.listdir(openPath)

#identifying genotypes
responder = "ACR"
driver = "MB082C"

wt = "w1118"
transgenic = driver + " x " + responder
filename = openPath + transgenic + ".csv"
filenamewt = openPath + wt+"_"+ transgenic + ".csv"

print(filename)
print(filenamewt)

D:\ACC Lab Dropbox\ACC Lab\Nicole Lee\Data Compilation\Falling_New\MB082C x ACR.csv
D:\ACC Lab Dropbox\ACC Lab\Nicole Lee\Data Compilation\Falling_New\w1118_MB082C x ACR.csv


In [13]:
dfe=pd.read_csv(filename)
dfw= pd.read_csv(filenamewt)

exptdf = dfe.drop(dfe.columns[[0]],axis = 1)
wtdf = dfw.drop(dfw.columns[[0]],axis = 1)

dfexpt = NLCLIMB.generation(exptdf, driver)
dfwt = NLCLIMB.generation(wtdf, wt)

In [14]:
df_sp = NLMATH.ospeed(dfwt, dfexpt).reset_index(drop=True)
dfs2 = NLMATH.deltaversion(df_sp, driver, "Velocity")

In [15]:
dfs2

,MBON,WT,Expt,delta_g
MB082C,MB082C,0.413,0.673,-0.091


In [16]:
df6 = df_sp[(df_sp['ExperimentState'] != "Recovery") ]       
name = []
if any(df6["Velocity"].isnull()):
    name = df6[df6["Velocity"].isnull()]['index'].tolist()
dfsp_db = df6[~df6['index'].isin(name)]
    
dfsp_db2 = dabest_jck.load(data = dfsp_db, x = ["ExperimentState", "Type"], y = "Velocity",  delta2 = True, experiment = "Type",
                        experiment_label = ['WT', 'Expt'], x1_level = ["Dark", "Full"], paired = "baseline", id_col="index" ) #if delta2 = dabest; deltaG = dabest_jck
dfstatstest = dfsp_db2.delta_g.statistical_tests